In [1]:
import numpy as np
import pandas as pd

data_path = 'YOUR_DATASET_PATH'
df_train = pd.read_csv(data_path + 'train.csv', nrows=500000)
df_test = pd.read_csv(data_path + 'test.csv')
train_Y = np.array(df_train['fare_amount'])
ids = df_test['key']

/var/folders/r6/7l4wrzxn2b1536z06d9p994m0000gn/T/ipykernel_9367/1593917178.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df_train = df_train.drop(['key', 'fare_amount'] , axis=1)
df_test = df_test.drop(['key'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()
df['distance_2D'] = ((df['dropoff_longitude'] - df['pickup_longitude'])**2 + (df['dropoff_latitude'] - df['pickup_latitude'])**2)**0.5
df = df.drop(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'pickup_datetime'], axis=1)
df.head()

,passenger_count,distance_2D
0,1,0.009436
1,1,0.079696
2,2,0.013674
3,1,0.025340
4,1,0.019470


In [3]:
df['passenger_count'] = df['passenger_count'].fillna(-1)
df['distance_2D'] = df['distance_2D'].fillna(-1)

In [4]:
train_num = train_Y.shape[0]
train_X = df[:train_num]
test_X = df[train_num:]

from sklearn import datasets, metrics
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor()
reg.fit(train_X, train_Y)
pred = reg.predict(test_X)

In [ ]:
import os

pred = np.array(pred)
sub = pd.DataFrame({'key': ids, 'fare_amount': pred})

if not os.path.exists('./baseline'):
    os.mkdir('./baseline')
sub.to_csv('./baseline/taxi_baseline.csv', index=False)